# Los Angeles restaurants: inspections and violations

This is a visualization of Los Angeles restaurant inspections and violations between 07/01/2015 and 12/29/2017. 

I've taken the two datasets found [here](https://www.kaggle.com/meganrisdal/la-county-restaurant-inspections-and-violations), joined them, and augmented them with geocode information.  With this info, we'll be able to plot a heat map with time.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import ast
import time
import datetime

import folium
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime

sns.set()
sns.set_palette("Set2")

plt.rcParams['figure.figsize'] = 8, 8
%matplotlib inline

In [2]:
data = pd.read_csv('violations_locations.csv')

In [3]:
def datest_to_timestamp(series):
    res = []
    for st in series:
        res.append(datetime.datetime.strptime(st, "%Y-%m-%d"))
    return res

In [4]:
data['Date'] = data['activity_date'].transform(datest_to_timestamp)

In [5]:
all_dates = data['Date'].values
all_dates = list(set(all_dates))
all_dates.sort()
time_labels = [str(x)[:10] for x in all_dates]

In [6]:
clean = data[(~data['lon'].isnull()) & (~data['lat'].isnull())]

In [7]:
heat_data = [[[row['lat'], row['lon'], row['points']] for index, row in clean[clean['Date'] == i].iterrows()] for i in all_dates]

In [8]:
def heat_map_time(start_date, end_date):
    # min_day is the date representation of the first index of the data array
    min_day = datetime.datetime.strptime(time_labels[0], "%Y-%m-%d")
    s_day = datetime.datetime.strptime(start_date, "%m/%d/%Y")
    e_day = datetime.datetime.strptime(end_date, "%m/%d/%Y")
    
    # the number of days relative to min_day are the indices in the data array we want
    s_idx = (s_day-min_day).days
    e_idx = (e_day-min_day).days
    
    # the data and times passed to HeatMapWithTime
    d = heat_data[s_idx:e_idx+1]
    times = time_labels[s_idx:e_idx+1]
    #the Stamen Toner tile looks cool
    hmaptime = folium.Map(location=[34, -118], tiles='Stamen Toner',zoom_start=10)
    
    h = HeatMapWithTime(data=d, index=times, use_local_extrema=True).add_to(hmaptime)
    h.save('heatmap.html')
    display(hmaptime)

In [9]:
heat_map_time('01/12/2017','02/11/2017')